# Solving-Hamilton-Jacobi-Bellman Equations
#### Frederik Kelbel, Imperial College London

## Dependencies

In [1]:
import torch
import plotly.graph_objects as go
import numpy as np
from operators import div, Δ, D
from DGM import DGMSolver, DGMPIASolver
from pdes import HBJ
from scipy.integrate import quad
from plotly.subplots import make_subplots
from configs import CONFIG_HBJS as MODEL_CONFIG

## Plotting

In [2]:
def plot_losses(losses, avg_over=10):
    avgs_1 = np.convolve(losses[:, 0], np.ones(avg_over), 'valid') / avg_over
    avgs_2 = np.convolve(losses[:, 1], np.ones(avg_over), 'valid') / avg_over
    fig = make_subplots(rows=1, cols=1)
    fig.add_trace(go.Scatter(x=np.arange(len(avgs_1)), y=avgs_1, mode='lines', name="Value Loss"), row=1, col=1)
    fig.add_trace(go.Scatter(x=np.arange(len(avgs_2)), y=avgs_2, mode='lines', name="Control Loss"), row=1, col=1)
    fig.update_layout(
        title="Loss",
        xaxis_title="Iterations",
        yaxis_title="Loss",
        font=dict(
            family="Courier New, monospace",
            size=14
        )
    )
    fig.show()
    
def plot_value(solver, sol):
    fig = make_subplots(rows=1, cols=2, 
                   specs=[[{'type': 'surface'}, {'type': 'surface'}]])
    xs = np.linspace(0, 1, 100)
    ts = np.linspace(0.01, 1, 100)
    us_pred = np.array([[solver(x, t) for x in xs] for t in ts])
    us = np.array([[sol(x, t) for x in xs] for t in ts])
    x_mesh, t_mesh = np.meshgrid(xs, ts)
    fig.add_trace(go.Surface(z=us, showscale=False), row=1, col=1)
    fig.add_trace(go.Surface(z=us_pred), row=1, col=2)
    fig.update_layout(title='Solution | Approximation',
                  scene = dict(
                    xaxis_title="t",
                    yaxis_title="x",
                    zaxis_title="J(x, t"),
                  scene2 = dict(
                    xaxis_title="t",
                    yaxis_title="x",
                    zaxis_title="J(x, t)"),
                  margin=dict(l=50, r=50, b=50, t=50))
    fig.show()
    
def plot_control(solver, sol):
    fig = make_subplots(rows=1, cols=1)
    eval_points = np.linspace(0.01, 1, 100)
    fig.add_trace(go.Scatter(x=eval_points, y=[solver(p) for p in eval_points], mode='lines', name="Optimal Control"), row=1, col=1)
    fig.add_trace(go.Scatter(x=eval_points, y=[sol(p) for p in eval_points], mode='lines', name="Optimal Control Solution"), row=1, col=1)
    fig.update_layout(
        title="Solutions",
        xaxis_title="Time",
        yaxis_title="Control Signal",
        yaxis_range=[-1,1],
        font=dict(
            family="Courier New, monospace",
            size=14
        )
    )
    fig.show()

## Problem Formulation

Objective: Find the control process $u = (u_t)_{t \geq 0}$ in admissable set $\mathcal{A}$ for an Itô Process $X^u = (X_t^u)_{t \geq 0}$ satisfying:


$$
d X_t^u = \mu(t, X_t^u, u_t) dt + \sigma(t, X_t^u, u_t) d W_t, \quad X_0^u = 0.
$$

We will consider the HBJ-Equations in their primal form.

The agents performance is assessed via:
$$
J^u(t, x) = \mathbb{E}\Big[ \int_t^T F(s, X_s^u, u_s) ds + G(X_T^u) \;\Big|\; X_t^u = x \Big]
$$

Denote $J(t, x) = \sup_{u \in \mathcal{A}} J^u(t, x)$, then this value function satisfies the following HJB-equations:

$$
\begin{cases}
\partial_t J(t, x) + \sup_{u \in \mathcal{A}} \{\mathscr{L}^u_t J(t, x) + F(t, x, u)\} = 0 \\
J(T, x) = G(x)
\end{cases}
$$

### The Merton Problem (Consumption Optimization)

The goal is to find the optimal wealth consumption strategy over time such that the consumption is maximized.

Let $(\Omega, \mathcal{F}, \{\mathcal{F}_t\}_{t\in [0, T]}, \mathbb{P})$ be a filtered probability space. The evolution of an investor's wealth is described via
$$
\begin{cases}
dX_s = ((\mu -r)u_s + r)X_s ds + \sigma u_s X_s dW_s, \; s \in [t, T] \\
X_0 = x > 0
\end{cases},
$$

with $\mu$ and $\sigma$ referring to drift and volatility, respectively. Let $r$ denote the discount rate, i.e. the depreciation constant. The intend is to maximise the objective
$$
J^c(t, X_t) = \mathbb{E}[ x^\gamma ], \; \gamma \in(0, 1).
$$
$u_t$ describes the fraction of the wealth to be invested into the risky asset.

The respective HBJ-equation becomes:

$$
\begin{cases}
\partial_t J(t, x) + \sup_{u} \Big\{ ((\mu-r)u + r)x \partial_x J(t, x) + \frac{1}{2} \sigma^2 u^2 x^2\partial_{xx} J(t, x) \Big\} = 0
\\
J(T, x) = x^\gamma
\end{cases}
$$

#### Analytical Solution:

Assume $J(t, x) = w(t) v(x)$, with $J(T, x) = w(T) v(x) = x^\gamma$. We guess $J(t, x) = w(t) x^\gamma$, where $w(T)=1$. The problem becomes 
$$
\begin{cases}
w'(t) + \gamma  \sup_{u} \Big\{ (\mu-r) u + r + \frac{1}{2} \sigma^2 u^2 (\gamma-1) \Big\}w(t) = 0
\\
w(T) = x^\gamma
\end{cases}.
$$

Then, $u$ is maximised for $u^* = \frac{\mu-r}{\sigma^2 (1-\gamma)}$ and the equation becomes
$$
\begin{cases}
w'(t) + \frac{\gamma (\mu-r)^2}{\sigma^2(2-2\gamma)}w(t) = 0
\\
w(T) = x^\gamma
\end{cases}.
$$
Thus, we have  $w'(t) = \frac{\gamma (\mu-r)^2}{\sigma^2(2\gamma - 2)}w(t)$. It follows that $w(t) = \exp\big(\frac{\gamma (\mu-r)^2}{\sigma^2(2\gamma - 2)}t\big)$. The final solution is:
$$
J(t, x) = w(t)v(x) = \exp\big(\frac{\gamma (\mu-r)^2}{\sigma^2(2\gamma - 2)}t\big) x^\gamma.
$$

We need to verify this using the HBJ-verification theorem.

In [13]:
class RISKY_ASSET(HBJ):
    def __init__(self):
        super().__init__()
        self.μ = 0.04
        self.σ = 0.4
        self.r = 0.03
        self.γ = 0.5
        
        def terminal_func(var):
            var[-1] = 0*var[-1] + 1
            return var
        
        self.var_dim = 2 # (x, t)
        self.cost_function = lambda u, var, t: 0
        self.differential_operator = lambda J, u, var, t: ((self.μ-self.r)*u+self.r)*var[0]*div(J, var[0]) + 0.5*self.σ**2*u**2*var[0]**2*Δ(J, var[0])
        self.domain_func = [lambda var: var]
        self.boundary_cond_J = [lambda J, var: J - var[0]**self.γ]
        self.boundary_func_J = [lambda var: terminal_func(var)]
        self.boundary_cond_u = [lambda u, var: torch.clamp(u, min=1) - torch.clamp(u, max=-1) - 2]
        self.boundary_func_u = [lambda var: var]

In [14]:
eq = RISKY_ASSET()
model = MODEL_CONFIG
solver = DGMPIASolver(model, eq)
loss = np.array(list(solver.train(800)))
plot_losses(loss)

100%|██████████| 800/800 [00:40<00:00, 19.61 it/s]


In [15]:
u_sol = lambda t : (eq.μ-eq.r)/(eq.σ**2*(1-eq.γ))
J_sol = lambda x, t: x**eq.γ * np.exp((eq.γ*(eq.μ-eq.r)**2)/(eq.σ**2*(2*eq.γ-2))*t)

In [16]:
plot_control(solver.u, u_sol)
plot_value(solver.J, J_sol)